## Chapter 20 Reframe Time Series Forecasting Problems

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
series = pd.read_csv('data/daily-minimum-temperatures.csv', index_col=0, parse_dates=True)
series.head()

,Temp
Date,
1981-01-01,20.7
1981-01-02,17.9
1981-01-03,18.8
1981-01-04,14.6
1981-01-05,15.8


#### Regression Framings

Most time series prediction problems are regression problems, requiring the prediction of a real-valued output. Below are 5 diﬀerent ways that this prediction problem could be re-phrased as an alternate regression problem:
- Forecast the change in the minimum temperature compared to the previous day.
- Forecast the minimum temperature relative to the average from the past 14 days.
- Forecast the minimum temperature relative to the average the same month last year.
- Forecast the minimum temperature rounded to the nearest 5 degrees Celsius.
- Forecast the average minimum temperature for the next 7 days.

Below is an example of reframing the Minimum Daily Temperatures forecast problem to predict the daily temperature rounded to the nearest 5 degrees.

In [16]:
# create a lagged dataset
values = pd.DataFrame(series.values)
df = pd.concat([values.shift(1), values], axis=1)
df.columns = ['t', 't+1']
# round forecast to nearest 5
for i in range(len(df)):
  df.loc[i, 't+1'] = int(df.loc[i, 't+1'] / 5) * 5.0
df.head()

,t,t+1
0,NaN,20.0
1,20.7,15.0
2,17.9,15.0
3,18.8,10.0
4,14.6,15.0


#### Classification Framings

Below are 5 diﬀerent ways that this prediction problem can be rephrased as a classification problem:
- Forecast whether a minimum temperature will be cold, moderate, or warm.
- Forecast whether a change in minimum temperature will be small or large.
Classification Framings
- Forecast whether the minimum temperature will be a monthly minimum or not.
- Forecast whether the minimum will be higher or lower than the minimum in the previous year.
- Forecast whether the minimum temperature will rise or fall over the next 7 days.

Below is an example of transforming the Minimum Daily Temperatures forecast problem to a classification problem where each temperature value is an ordinal value of cold, moderate, or hot. These labels are mapped to integer values, defined as:
- 0 (cold): < 10 degress Celsius
- 1 (moderate): >= 10 and < 25 degress Celsius
- 2 (hot): >= 25 degress Celsius

In [17]:
# reframe regression as classificiation
# create lagged dataset
values = pd.DataFrame(series.values)
df = pd.concat([values.shift(1), values], axis=1)
df.columns = ['t', 't+1']
# make discrete
for i in range(len(df)):
  value = df.loc[i, 't+1']
  if value < 10.0:
    df.loc[i, 't+1'] = 0
  elif value >= 25.0:
    df.loc[i, 't+1'] = 2
  else:
    df.loc[i, 't+1'] = 1
df.head()

,t,t+1
0,NaN,1.0
1,20.7,1.0
2,17.9,1.0
3,18.8,1.0
4,14.6,1.0


#### Time Horizon Framings

Another axis that can be varied is the time horizon. The time horizon is the number of time steps in the future that are being predicted. Below are 5 diﬀerent ways that this prediction problem can be re-phrased as a diﬀerent time horizon:
- Forecast the minimum temperature for the next 7 days.
- Forecast the minimum temperature in 30 days time.
- Forecast the average minimum temperature next month.
- Forecast the day in the next week that will have the lowest minimum temperature.
- Forecast one year of minimum temperature values.

The example below transforms the Minimum Daily Temperatures forecast problem to predict the minimum temperature for the next 7 days.
The problem is defined as: given the minimum daily temperature from the day before in degrees Celsius, forecast the minimum daily temperature for the next 7 days.

In [18]:
# create lagged dataset
values = pd.DataFrame(series.values)
df = pd.concat([values.shift(1), values, values.shift(-1), values.shift(-2), values.shift(-3), values.shift(-4), values.shift(-5), values.shift(-6)], axis=1)
df.columns = ['t', 't+1', 't+2', 't+3', 't+4', 't+5', 't+6', 't+7']
df.head(14)

,t,t+1,t+2,t+3,t+4,t+5,t+6,t+7
0,NaN,20.7,17.9,18.8,14.6,15.8,15.8,15.8
1,20.7,17.9,18.8,14.6,15.8,15.8,15.8,17.4
2,17.9,18.8,14.6,15.8,15.8,15.8,17.4,21.8
3,18.8,14.6,15.8,15.8,15.8,17.4,21.8,20.0
4,14.6,15.8,15.8,15.8,17.4,21.8,20.0,16.2
5,15.8,15.8,15.8,17.4,21.8,20.0,16.2,13.3
6,15.8,15.8,17.4,21.8,20.0,16.2,13.3,16.7
7,15.8,17.4,21.8,20.0,16.2,13.3,16.7,21.5
8,17.4,21.8,20.0,16.2,13.3,16.7,21.5,25.0
9,21.8,20.0,16.2,13.3,16.7,21.5,25.0,20.7
